<a href="https://colab.research.google.com/github/berrygayo/BigdataAnalysisCertification/blob/main/%ED%9A%8C%EA%B7%80_%EB%B6%84%EB%A5%98%EC%A0%95%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 회귀 

# 제주빅데이터센터 데이터를 활용하여 효율적인 교통량 측정을 위한 날씨/유동인구 활용 교통량 추이 데이터 제공 

In [ ]:
# 0050 start
# 0148 fin 
# ==============
#  preprocessing
# 라벨인코딩(시도명,읍면도명),id 제거 
# ==============

import pandas as pd
train_X = pd.read_csv("https://raw.githubusercontent.com/Datamanim/jeju/main/Jeju_trainX.csv",encoding='euc-kr')
train_y= pd.read_csv("https://raw.githubusercontent.com/Datamanim/jeju/main/Jeju_trainy.csv",encoding='euc-kr')
test_X= pd.read_csv("https://raw.githubusercontent.com/Datamanim/jeju/main/Jeju_testX.csv",encoding='euc-kr')
sub= pd.read_csv("https://raw.githubusercontent.com/Datamanim/jeju/main/subExample.csv",encoding='euc-kr')


from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, BaggingRegressor, AdaBoostRegressor 
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.model_selection import GridSearchCV, cross_val_score,train_test_split
from sklearn.pipeline import Pipeline, make_pipeline 
from  sklearn.preprocessing import LabelEncoder,MinMaxScaler, RobustScaler, StandardScaler

#get_summies 확인필요 !!!!!!!!!!!!!!!!! 
train_X['시도명'] = LabelEncoder().fit_transform(train_X['시도명'])
train_X['읍면동명'] = LabelEncoder().fit_transform(train_X['읍면동명'])
test_X['시도명'] = LabelEncoder().fit_transform(test_X['시도명'])
test_X['읍면동명'] =LabelEncoder().fit_transform(test_X['읍면동명'])

# 월만 빼오기 
train_X['일자'] = pd.to_datetime(train_X['일자'])
train_X['월'] = train_X['일자'].dt.month
test_X['일자'] = pd.to_datetime(test_X['일자'])
test_X['월'] = test_X['일자'].dt.month

train_X = train_X.drop(['id','일자'], axis=1)
test_X = test_X.drop(['id','일자'], axis=1)
train_y = train_y.drop('id', axis=1)

sc = StandardScaler()
train_X[['거주인구','근무인구','방문인구','총 유동인구']] = sc.fit_transform(train_X[['거주인구','근무인구','방문인구','총 유동인구']])
test_X[['거주인구','근무인구','방문인구','총 유동인구']] = sc.fit_transform(test_X[['거주인구','근무인구','방문인구','총 유동인구']])

# ================
# 모델찾기 
# ===============

"""
RF = RandomForestRegressor()
GBR = GradientBoostingRegressor()
BR = BaggingRegressor()
ABR = AdaBoostRegressor()
LR = LinearRegression()
Lasso =  Lasso()
Ridge = Ridge()

X_train, X_val, y_train, y_val = train_test_split(train_X, train_y, test_size=0.15)

print('>>> Randomforest')
RF.fit(X_train, y_train)
print(RF.score(X_train, y_train))
print(RF.score(X_val, y_val))

print('>>> GradientBoostingRegressor')
GBR.fit(X_train, y_train)
print(GBR.score(X_train, y_train))
print(GBR.score(X_val, y_val))

print('>>> BaggingRegressor')
BR.fit(X_train, y_train)
print(BR.score(X_train, y_train))
print(BR.score(X_val, y_val))

print('>>> AdaBoostRegressor')
ABR.fit(X_train, y_train)
print(ABR.score(X_train, y_train))
print(ABR.score(X_val, y_val))

print('>>> LinearRegression')
LR.fit(X_train, y_train)
print(LR.score(X_train, y_train))
print(LR.score(X_val, y_val))

print('>>> Lasso')
Lasso.fit(X_train, y_train)
print(Lasso.score(X_train, y_train))
print(Lasso.score(X_val, y_val))

print('>>> Ridge')
Ridge.fit(X_train, y_train)
print(Ridge.score(X_train, y_train))
print(Ridge.score(X_val, y_val))
"""


# ================= 
# 하이퍼파라미터 튜닝 
# =================
"""
RF = RandomForestRegressor()
GBR = GradientBoostingRegressor()
BR = BaggingRegressor()

####### 일단 검정 셋으로 확인할거라 X_train 사용 > train_X로 변경 해야함 !!! 
print('>>> Randomforest')
# {'max_depth': 7, 'n_estimators': 10} 
# 0.87

param_grid = {'max_depth':[3,5,7],'n_estimators':[1,10,20]}
gs = GridSearchCV(RF,param_grid=param_grid, cv=5, verbose=True)
gs.fit(X_train, y_train)
print(gs.best_params_)
print(gs.best_score_)

print('>>> GradientBoostingRegressor')
#{'learning_rate': 0.1, 'n_estimators': 200}
# 0.9488039085087507
param_grid = {'learning_rate':[0.01,0.1,1],'n_estimators':[10,100,200]}
gs = GridSearchCV(GBR,param_grid=param_grid, cv=5, verbose=True)
gs.fit(X_train, y_train)
print(gs.best_params_)
print(gs.best_score_)

print('>>> BaggingRegressor')
#{'n_estimators': 20}
#0.9674245574047695
param_grid = {'n_estimators':[1,10,20]}
gs = GridSearchCV(BR,param_grid=param_grid, cv=5, verbose=True)
gs.fit(X_train, y_train)
print(gs.best_params_)
print(gs.best_score_)
"""
# ========= 
# 모델 생성 
# ======== 
RF = RandomForestRegressor(max_depth= 7, n_estimators= 10)
RF.fit(train_X, train_y)
pred_RF = RF.predict(test_X)

GBR = GradientBoostingRegressor(learning_rate= 0.1, n_estimators= 200)
GBR.fit(train_X, train_y)
pred_GBR = GBR.predict(test_X)

BR = BaggingRegressor(n_estimators=20)
BR.fit(train_X, train_y)
pred_BR = BR.predict(test_X)

pred_y = (pred_RF+pred_GBR+pred_BR)/3

# 분류

In [ ]:
# 3:20 start 

import pandas as pd 
import numpy as np 
import sklearn
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split
from sklearn.pipeline import Pipeline, make_pipeline 
from sklearn.preprocessing import LabelEncoder, RobustScaler, StandardScaler, MinMaxScaler 
from sklearn.metrics import accuracy_score, make_scorer, roc_auc_score
pd.set_option('display.max_columns',500)

X_train = pd.read_csv('data/X_train.csv')
X_test = pd.read_csv('data/X_test.csv')
y_train = pd.read_csv('data/y_train.csv')

# ==================
# preprocessing X_train, X_test 동일하게 해야함 ! 
# 1. 결측값 확인, 최대최솟값 확인, 라벨인코딩, 마지막에 cust_id 제거 
# X_train : 3500개, X_test :2500개 
# ==================== 
X_train = X_train.fillna(0)
X_test = X_test.fillna(0)

X_train['주구매상품'] = pd.get_dummies(X_train['주구매상품'])
X_train['주구매지점'] = pd.get_dummies(X_train['주구매지점'])
X_test['주구매상품'] = pd.get_dummies(X_test['주구매상품'])
X_test['주구매지점'] = pd.get_dummies(X_test['주구매지점'])

X_train = X_train.drop('cust_id', axis=1)
X_test = X_test.drop('cust_id', axis=1)
y_train = y_train.drop('cust_id', axis=1)

# ================
# 모델선택 
# ===============
LR = Pipeline([('sc',RobustScaler()),('model',LogisticRegression())])#**************수케일러,모델 뒤 소괄호 
RF = Pipeline([('sc',RobustScaler()),('model',RandomForestClassifier())])
Bag = Pipeline([('sc',RobustScaler()),('model',BaggingClassifier())])
GBR = Pipeline([('sc',RobustScaler()),('model',GradientBoostingClassifier())])

print("LR : ", cross_val_score(LR,X_train,y_train,cv=5,verbose=True).mean()) #0.634
print("RF : ", cross_val_score(RF,X_train,y_train,cv=5,verbose=True).mean()) 
print("Bag : ", cross_val_score(Bag,X_train,y_train,cv=5,verbose=True).mean())
print("GBR : ", cross_val_score(GBR,X_train,y_train,cv=5,verbose=True).mean()) #0.631

# =========== 
# 하이퍼파라미터튜닝 
# ========== 
score = make_scorer(roc_auc_score)
# 랜덤 포레스트 : {4,10}
RF = Pipeline([('sc',RobustScaler()),('model',RandomForestClassifier())])
param_grid = {'model__max_depth':[2,3,4], 'model__n_estimators':[10,100,200]} # ****model__ 붙여야함 !!!  
gs = GridSearchCV(RF,param_grid,scoring=score,cv=5)
gs.fit(X_train,y_train)
print(gs.best_params_)
print(gs.best_score_)

# GradientBoosting {0.1,100}
GBR = Pipeline([('sc',RobustScaler()),('model',GradientBoostingClassifier())])
param_grid = {'model__learning_rate':[0.01,0.1,1], 'model__n_estimators':[10,100,200]} # ****model__ 붙여야함 !!!  
gs = GridSearchCV(GBR,param_grid,scoring=score,cv=5)
gs.fit(X_train,y_train)
print(gs.best_params_)
print(gs.best_score_)

# Logistic {1}
LR = Pipeline([('sc',RobustScaler()),('model',LogisticRegression())])
param_grid = {'model__C':[0.1,1,10]} # ****model__ 붙여야함 !!!  
gs = GridSearchCV(LR,param_grid,scoring=score,cv=5)
gs.fit(X_train,y_train)
print(gs.best_params_)
print(gs.best_score_)

# ==========
# 모델 적용 
# =========== 
LR = LogisticRegression()
LR.fit(X_train,y_train)
pred_LR = LR.predict_proba(X_test)

GBR = GradientBoostingClassifier()
GBR.fit(X_train,y_train)
pred_GBR = GBR.predict_proba(X_test)

RF = RandomForestClassifier(max_depth=4, n_estimators=10)
RF.fit(X_train,y_train)
pred_RF = RF.predict_proba(X_test)

pred_y = (pred_LR + pred_RF + pred_GBR)/3
pred_Y= pd.DataFrame({'gender':pred_y[:,1]})

X_test_2 = pd.read_csv('data/X_test.csv')
res=pd.concat([X_test_2[['cust_id']], pred_Y],axis=1)

res.to_csv('14010353.csv')